<a href="https://colab.research.google.com/github/mrrajatgarg/my_project_works/blob/master/MultiLabelMusicAutoTagging2DCNNTrainingAndTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Music Information Retrieval (MIR)

**Defination --**

Music Information Retreval (MIR) is centered on finding ways to automate the process of classifying music genre and mood and tagging music.

**AIM:**

It's a multi-label classification problem (i.e. a particular audio file can have multiple tags associated with it. for ex: - indian, guitar). Given the audio clip, tag the audio clip with appropriate tags.

# Importing Libraries

In [0]:
import pandas as pd
import numpy as np

In [0]:
!pip install sklearn
import sklearn
from sklearn.model_selection import train_test_split

In [0]:
import keras
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras.utils import to_categorical

import kapre
from kapre.time_frequency import Melspectrogram
from kapre.utils import Normalization2D

Using TensorFlow backend.


# Loading the .npy file saved earlier from Google Drive

In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Link is the sharable link of the file

In [0]:
link = 'https://drive.google.com/open?id=1y3S1mYaB3kygbvhe_sqN0OZqwe6ANmaj'

fluff, id = link.split('=')

print (id) # Verify that you have everything after '='

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('list_data-file-duration29.npy') 

1y3S1mYaB3kygbvhe_sqN0OZqwe6ANmaj


In [0]:
list_data = np.load('list_data-file-duration29.npy')

I have loaded only 16250 audio files as loading more files causes my Google Colab to run out of memory.

In [0]:
list_data.shape

(16250, 1, 61394)

# Preprocessing on Labels of audio files for getting the similarity used by most researchers to publish betchmarks

Steps of Preprocessing includes:


*   Removal of similar labels to single category
*   Reducing the labels to 50 most common labels



In [0]:
directory_labels = '/content/'

In [0]:
# Load the annotations file (prior to execution check correctness of delimiter)
annotations = pd.read_csv(directory_labels + 'annotations_final.csv', sep="\t")

In [0]:
# check size of annotations
annotations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25863 entries, 0 to 25862
Columns: 190 entries, clip_id to mp3_path
dtypes: int64(189), object(1)
memory usage: 37.5+ MB


In [0]:
# ensure correct loading
annotations.head(5)

,clip_id,no voice,singer,duet,plucking,hard rock,world,bongos,harpsichord,female singing,...,rap,metal,hip hop,quick,water,baroque,women,fiddle,english,mp3_path
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
1,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
2,10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
3,11,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
4,12,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...


In [0]:
# merge synonym tags
synonyms = [['beat', 'beats'],
            ['chant', 'chanting'],
            ['choir', 'choral'],
            ['classical', 'clasical', 'classic'],
            ['drum', 'drums'],
            ['electro', 'electronic', 'electronica', 'electric'],
            ['fast', 'fast beat', 'quick'],
            ['female', 'female singer', 'female singing', 'female vocals', 'female vocal', 'female voice', 'woman', 'woman singing', 'women'],
            ['flute', 'flutes'],
            ['guitar', 'guitars'],
            ['hard', 'hard rock'],
            ['harpsichord', 'harpsicord'],
            ['heavy', 'heavy metal', 'metal'],
            ['horn', 'horns'],
            ['india', 'indian'],
            ['jazz', 'jazzy'],
            ['male', 'male singer', 'male vocal', 'male vocals', 'male voice', 'man', 'man singing', 'men'],
            ['no beat', 'no drums'],
            ['no singer', 'no singing', 'no vocal','no vocals', 'no voice', 'no voices', 'instrumental'],
            ['opera', 'operatic'],
            ['orchestra', 'orchestral'],
            ['quiet', 'silence'],
            ['singer', 'singing'],
            ['space', 'spacey'],
            ['string', 'strings'],
            ['synth', 'synthesizer'],
            ['violin', 'violins'],
            ['vocal', 'vocals', 'voice', 'voices'],
            ['strange', 'weird']]

In [0]:
# replace all synonym words with the first in the list constructed above
for synonym_list in synonyms:
    annotations[synonym_list[0]] = annotations[synonym_list].max(axis=1)
    annotations.drop(synonym_list[1:], axis=1, inplace=True)

In [0]:
# check results
# previously: Columns: 190 entries, clip_id to mp3_path &  memory usage: 37.5+ MB
# now: Columns: 136 entries, clip_id to mp3_path & memory usage: 26.8+ MB
annotations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25863 entries, 0 to 25862
Columns: 136 entries, clip_id to mp3_path
dtypes: int64(135), object(1)
memory usage: 26.8+ MB


In [0]:
# keep clip_id and mp3_path in a different matrix to bind later
annotations_ids = annotations[['clip_id', 'mp3_path']]

In [0]:
# drop clip_id and mp3_path to proceed with distribution counts
annotations = annotations[annotations.columns.difference(['clip_id', 'mp3_path'])]

In [0]:
annotations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25863 entries, 0 to 25862
Columns: 134 entries, acoustic to world
dtypes: int64(134)
memory usage: 26.4 MB


In [0]:
# tags' distribution
tags_distro = annotations.sum(axis=0)
# sort tags according to their occurencies 
tags_distro.sort_values(axis=0, inplace=True, ascending = False)

In [0]:
#find 50 top tags
topindex, topvalues = list(tags_distro.index[:50]), tags_distro.values[:50]

In [0]:
# list of columns to remove from annotation file (not common labels)
rem_cols_index =list(tags_distro.index[50:])

In [0]:
#check how many columns we are about to remove
len(rem_cols_index)

84

In [0]:
# keep only the 50 most common tags
annotations.drop(rem_cols_index, axis=1, inplace=True)

In [0]:
# ensure correct binding
annotations.head(5)

,ambient,baroque,bass,beat,cello,chant,choir,classical,country,dance,...,slow,soft,solo,strange,string,synth,techno,trance,violin,vocal
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
1,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0


In [0]:
annotations.shape

(25863, 50)

# Converting above files to format so that we can train model on them

In [0]:
print(list_data.shape)
print(annotations.shape)

(16250, 1, 61394)
(25863, 50)


In [0]:
new_annotations = annotations[0:16250]

In [0]:
classes = new_annotations.values

In [0]:
classes.shape

(16250, 50)

# Splitting of Dataset into training (80%) and validation (20%) set.

In [0]:
X_train, X_test, y_train, y_test = train_test_split(list_data, classes, test_size=0.2, random_state=1) #20000

In [0]:
X_train.shape

(13000, 1, 61394)

In [0]:
X_test.shape

(3250, 1, 61394)

# Building of CNN model

In [0]:
input_shape = X_train.shape[1:]  
input_shape

(1, 61394)

In [0]:
model = Sequential()
model.add(Melspectrogram(n_dft=1024, n_hop=512, input_shape=input_shape, return_decibel_melgram=True, name='melgram'))
model.add(Normalization2D(str_axis='freq'))

# Layer 1
model.add(Conv2D(32, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2))) 

# Layer 2
model.add(Conv2D(32, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Flatten()) 

model.add(Dense(100, activation='relu')) 
# model.add(BatchNormalization())
model.add(Dense(50,activation='sigmoid'))

model.compile(loss='binary_crossentropy' , optimizer='sgd' , metrics=['accuracy'])

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
melgram (Melspectrogram)     (None, 128, 120, 1)       1116288   
_________________________________________________________________
normalization2d_1 (Normaliza (None, 128, 120, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 126, 118, 32)      320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 59, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 61, 57, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 28, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 26880)             0         
__________

# Training of Model and checking Probabilistic Accuracy

In [0]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=10)

Train on 13000 samples, validate on 3250 samples
Epoch 1/10
13000/13000 [==============================] - 21s 2ms/step - loss: 0.2452 - acc: 0.9276 - val_loss: 0.1822 - val_acc: 0.9473
Epoch 2/10
13000/13000 [==============================] - 17s 1ms/step - loss: 0.1817 - acc: 0.9463 - val_loss: 0.1769 - val_acc: 0.9473
Epoch 3/10
13000/13000 [==============================] - 17s 1ms/step - loss: 0.1778 - acc: 0.9464 - val_loss: 0.1737 - val_acc: 0.9474
Epoch 4/10
13000/13000 [==============================] - 17s 1ms/step - loss: 0.1750 - acc: 0.9464 - val_loss: 0.1718 - val_acc: 0.9476
Epoch 5/10
13000/13000 [==============================] - 17s 1ms/step - loss: 0.1732 - acc: 0.9466 - val_loss: 0.1704 - val_acc: 0.9475
Epoch 6/10
13000/13000 [==============================] - 17s 1ms/step - loss: 0.1720 - acc: 0.9466 - val_loss: 0.1692 - val_acc: 0.9478
Epoch 7/10
13000/13000 [==============================] - 17s 1ms/step - loss: 0.1706 - acc: 0.9468 - val_loss: 0.1683 - val_acc:

# Checking Accuracy on More Relevant Metrics like AUC

In [0]:
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback

In [0]:
class roc_callback(Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]


    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.x)
        roc = roc_auc_score(self.y, y_pred)
        y_pred_val = self.model.predict(self.x_val)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        print('\rroc-auc: %s - roc-auc_val: %s' % (str(round(roc,4)),str(round(roc_val,4))),end=100*' '+'\n')
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [0]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), 
          callbacks=[roc_callback(training_data=(X_train, y_train),validation_data=(X_test, y_test))],
         batch_size=32, epochs=10)

Train on 13000 samples, validate on 3250 samples
Epoch 1/10
13000/13000 [==============================] - 20s 2ms/step - loss: 0.2523 - acc: 0.9227 - val_loss: 0.1840 - val_acc: 0.9473
roc-auc: 0.5885 - roc-auc_val: 0.5947                                                                                                    
Epoch 2/10
13000/13000 [==============================] - 15s 1ms/step - loss: 0.1840 - acc: 0.9463 - val_loss: 0.1788 - val_acc: 0.9474
roc-auc: 0.6329 - roc-auc_val: 0.6384                                                                                                    
Epoch 3/10
13000/13000 [==============================] - 15s 1ms/step - loss: 0.1798 - acc: 0.9463 - val_loss: 0.1754 - val_acc: 0.9474
roc-auc: 0.6617 - roc-auc_val: 0.6676                                                                                                    
Epoch 4/10
13000/13000 [==============================] - 16s 1ms/step - loss: 0.1769 - acc: 0.9464 - val_loss: 0.1733 - val_a

# Model 2

In [0]:
model2 = Sequential()
model2.add(Melspectrogram(n_dft=1024, n_hop=512, input_shape=input_shape, return_decibel_melgram=True, name='melgram'))
model2.add(Normalization2D(str_axis='freq'))

# Layer 1
model2.add(Conv2D(32, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2, 2))) 

# Layer 2
model2.add(Conv2D(32, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2, 2))) 

# Layer 3
model2.add(Conv2D(32, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Flatten()) 

model2.add(Dense(100, activation='relu')) 
# model.add(BatchNormalization())
model2.add(Dense(50,activation='sigmoid'))

model2.compile(loss='binary_crossentropy' , optimizer='sgd' , metrics=['accuracy'])

In [0]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
melgram (Melspectrogram)     (None, 128, 120, 1)       1116288   
_________________________________________________________________
normalization2d_4 (Normaliza (None, 128, 120, 1)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 126, 118, 32)      320       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 63, 59, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 61, 57, 32)        9248      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 30, 28, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 28, 26, 32)        9248      
__________

In [0]:
model2.fit(X_train, y_train, validation_data=(X_test, y_test), 
          callbacks=[roc_callback(training_data=(X_train, y_train),validation_data=(X_test, y_test))],
         batch_size=32, epochs=10)

Train on 13000 samples, validate on 3250 samples
Epoch 1/10
13000/13000 [==============================] - 16s 1ms/step - loss: 0.3031 - acc: 0.8972 - val_loss: 0.1935 - val_acc: 0.9473
roc-auc: 0.5256 - roc-auc_val: 0.5351                                                                                                    
Epoch 2/10
13000/13000 [==============================] - 16s 1ms/step - loss: 0.1917 - acc: 0.9463 - val_loss: 0.1850 - val_acc: 0.9473
roc-auc: 0.5784 - roc-auc_val: 0.5867                                                                                                    
Epoch 3/10
13000/13000 [==============================] - 16s 1ms/step - loss: 0.1857 - acc: 0.9463 - val_loss: 0.1806 - val_acc: 0.9473
roc-auc: 0.6146 - roc-auc_val: 0.6218                                                                                                    
Epoch 4/10
13000/13000 [==============================] - 16s 1ms/step - loss: 0.1818 - acc: 0.9463 - val_loss: 0.1777 - val_a

# Model 3

In [0]:
model3 = Sequential()
model3.add(Melspectrogram(n_dft=1024, n_hop=512, input_shape=input_shape, return_decibel_melgram=True, name='melgram'))
model3.add(Normalization2D(str_axis='freq'))

# Layer 1
model3.add(Conv2D(32, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Dropout(0.5))

# Layer 2
model3.add(Conv2D(32, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Dropout(0.5))

# Layer 3
model3.add(Conv2D(32, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Flatten()) 
model3.add(Dropout(0.5))

model3.add(Dense(100, activation='relu')) 
# model.add(BatchNormalization())
model3.add(Dense(50,activation='sigmoid'))

model3.compile(loss='binary_crossentropy' , optimizer='sgd' , metrics=['accuracy'])

In [0]:
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
melgram (Melspectrogram)     (None, 128, 120, 1)       1116288   
_________________________________________________________________
normalization2d_6 (Normaliza (None, 128, 120, 1)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 126, 118, 32)      320       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 63, 59, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 63, 59, 32)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 61, 57, 32)        9248      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 30, 28, 32)        0         
__________

In [0]:
model3.fit(X_train, y_train, validation_data=(X_test, y_test), 
          callbacks=[roc_callback(training_data=(X_train, y_train),validation_data=(X_test, y_test))],
         batch_size=32, epochs=10)

Train on 13000 samples, validate on 3250 samples
Epoch 1/10
13000/13000 [==============================] - 18s 1ms/step - loss: 0.2798 - acc: 0.9053 - val_loss: 0.3236 - val_acc: 0.9473
roc-auc: 0.5454 - roc-auc_val: 0.5439                                                                                                    
Epoch 2/10
13000/13000 [==============================] - 17s 1ms/step - loss: 0.2031 - acc: 0.9461 - val_loss: 0.3188 - val_acc: 0.9473
roc-auc: 0.5686 - roc-auc_val: 0.5662                                                                                                    
Epoch 3/10
13000/13000 [==============================] - 17s 1ms/step - loss: 0.1967 - acc: 0.9462 - val_loss: 0.3183 - val_acc: 0.9466
roc-auc: 0.5829 - roc-auc_val: 0.5805                                                                                                    
Epoch 4/10
13000/13000 [==============================] - 17s 1ms/step - loss: 0.1930 - acc: 0.9462 - val_loss: 0.3162 - val_a